In [1]:
import tensorflow as tf
import edward as ed
from edward.models import Beta

In [13]:
import tensorflow as tf
n = tf.Variable(tf.constant([6,5,3,3,2,1]))
table = tf.constant(2, shape=(1,))
sparse = tf.scatter_add(n, table, tf.constant([1]))
sess = tf.Session()
sess.run(n.initializer)
sess.run(sparse)

array([6, 5, 4, 3, 2, 1], dtype=int32)

In [26]:
def indicator(idx, length):
    x = tf.ones((1,))
    if idx == 0:
        return tf.concat([x, tf.zeros(length-1)], 0)
    elif idx == length - 1:
        return tf.concat([tf.zeros(length-1), x], 0)
    paddings = tf.concat([[idx], [length - idx - 1]], 0)
    return tf.pad(x, [paddings])
x = tf.constant((1,2,3,4,5,6))

sess = tf.Session()
for i in range(6):
    print(sess.run(indicator(i, x.shape[0])))

ValueError: Operation 'concat_29' has been marked as not fetchable.

In [23]:
class ChineseRestaurantProcess(object):
    
    def __init__(self, alpha):
        # The table sizes, initially just an empty table of size alpha
        self.n = tf.reshape(alpha, (1,))

    def alpha(self):
        "The concentration parameter of the process."
        return self.n[0]

    def occupied_tables(self):
        "The number of occupied tables."
        return self.n.shape[0] - 1  # Take one off for the initial alpha
    
    def sample(self):
        # Sample a table
        table = tf.reshape(
            tf.multinomial(logits=tf.reshape(tf.log(self.n), [1, -1]), num_samples=1),
            (1,))
        
        def sit_at_existing():
            print(self.n.shape)
            print(self.n.shape[0])
            return self.n + indicator(table, self.n.shape[0])
            
        # Are we creating a new table?
        cond =  tf.equal(tf.zeros((), dtype=tf.int64), tf.reshape(table, ()))
        self.n = tf.cond(cond, self._sit_at_new, sit_at_existing)
        return self.n
        
    def _sit_at_new(self):
        return tf.concat([self.n, tf.ones((1,))], 0)

crp = ChineseRestaurantProcess(.1)
for i in range(5):
    print('Sampling')
    s = crp.sample()

sess = tf.Session()
sess.run(s)

Sampling
(1,)
1


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'